In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import math

In [36]:
# Dataframe, Column, Quantile
def outliers(df,minq,maxq,col):
    min_v = df.quantile(minq)[col]
    max_v = df.quantile(maxq)[col]
    df1 = df.loc[df[col] > max_v]
    df2 = df.loc[df[col] < min_v]   
    return pd.concat((df1,df2))[col]

In [37]:
def Winsorizing(df,minq,maxq,col):
    min_v = df.quantile(minq)[col]
    max_v = df.quantile(maxq)[col]
    df[col] = df[col].apply(lambda x: max_v if x > max_v else x)
    df[col] = df[col].apply(lambda x: min_v if x < min_v else x)

# Loading data

In [38]:
train = pd.read_csv('train.csv')
train = train[train.IsOpen != 0]
target = train['NumberOfSales']
test = pd.read_csv('test.csv')

In [39]:
all_data_index = train.columns.drop(['NumberOfSales','NumberOfCustomers'])
all_data = pd.concat((train.loc[:,all_data_index],test.loc[:,all_data_index]))
test_offset = train.shape[0]
all_data.columns

Index(['StoreID', 'Date', 'IsHoliday', 'IsOpen', 'HasPromotions', 'StoreType',
       'AssortmentType', 'NearestCompetitor', 'Region', 'Region_AreaKM2',
       'Region_GDP', 'Region_PopulationK', 'CloudCover', 'Events',
       'Max_Dew_PointC', 'Max_Gust_SpeedKm_h', 'Max_Humidity',
       'Max_Sea_Level_PressurehPa', 'Max_TemperatureC', 'Max_VisibilityKm',
       'Max_Wind_SpeedKm_h', 'Mean_Dew_PointC', 'Mean_Humidity',
       'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Mean_VisibilityKm',
       'Mean_Wind_SpeedKm_h', 'Min_Dew_PointC', 'Min_Humidity',
       'Min_Sea_Level_PressurehPa', 'Min_TemperatureC', 'Min_VisibilitykM',
       'Precipitationmm', 'WindDirDegrees'],
      dtype='object')

## Analyzing Features

In [40]:
my_numerical_variables = (pd.Index(['Min_Dew_PointC', 'Min_Humidity', 'Min_Sea_Level_PressurehPa','Min_TemperatureC','Min_VisibilitykM','Precipitationmm','WindDirDegrees']))

Precence of missing values

In [41]:
ratios = (all_data.isnull().sum() / len(all_data))*100
rate = ratios.drop(ratios[ratios == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :rate})
missing_data

,Missing Ratio
Max_Gust_SpeedKm_h,78.358042
Events,25.039665
CloudCover,8.855471
Min_VisibilitykM,2.476613
Mean_VisibilityKm,2.476613
Max_VisibilityKm,2.476613


In [42]:
len(all_data.loc[np.isnan(all_data['Min_VisibilitykM'])]['Mean_VisibilityKm'])

11879

In [43]:
len(all_data.loc[np.isnan(all_data['Min_VisibilitykM'])]['Max_VisibilityKm'])

11879

In [44]:
mean_of_min = math.ceil(all_data.loc[~np.isnan(all_data['Min_VisibilitykM'])]['Min_VisibilitykM'].describe()['mean'])

In [45]:
mean_of_max = math.ceil(all_data.loc[~np.isnan(all_data['Max_VisibilityKm'])]['Max_VisibilityKm'].describe()['mean'])

In [46]:
mean_of_mean = math.ceil(all_data.loc[~np.isnan(all_data['Max_VisibilityKm'])]['Max_VisibilityKm'].describe()['mean'])

In [47]:
all_data['Max_VisibilityKm'] = all_data['Max_VisibilityKm'].fillna(mean_of_max)
all_data['Min_VisibilitykM'] = all_data['Min_VisibilitykM'].fillna(mean_of_min)
all_data['Mean_VisibilityKm'] = all_data['Mean_VisibilityKm'].fillna(mean_of_mean)
ratios = (all_data.isnull().sum() / len(all_data))*100
rate = ratios.drop(ratios[ratios == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :rate})
missing_data

,Missing Ratio
Max_Gust_SpeedKm_h,78.358042
Events,25.039665
CloudCover,8.855471


In [48]:
all_data.iloc[[test_offset]]   #get row by index, here i verified that the date are above train and below test so they can be splitted
                               #X_test = all_data[train.shape[0]:]

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,AssortmentType,NearestCompetitor,Region,Region_AreaKM2,...,Mean_TemperatureC,Mean_VisibilityKm,Mean_Wind_SpeedKm_h,Min_Dew_PointC,Min_Humidity,Min_Sea_Level_PressurehPa,Min_TemperatureC,Min_VisibilitykM,Precipitationmm,WindDirDegrees
0,1000,01/03/2018,0,1,0,Hyper Market,General,326,7,9643,...,6,14.0,5,-1,46,1011,2,10.0,0.0,180


In [49]:
test.iloc[[0]]

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,AssortmentType,NearestCompetitor,Region,Region_AreaKM2,...,Mean_TemperatureC,Mean_VisibilityKm,Mean_Wind_SpeedKm_h,Min_Dew_PointC,Min_Humidity,Min_Sea_Level_PressurehPa,Min_TemperatureC,Min_VisibilitykM,Precipitationmm,WindDirDegrees
0,1000,01/03/2018,0,1,0,Hyper Market,General,326,7,9643,...,6,14.0,5,-1,46,1011,2,10.0,0.0,180


## Converting each date in int

In [50]:
months = []
all_date = all_data['Date']
for s in all_date:
    months.append(int(s[3:5]))

## Adding the new feature to all data

In [51]:
all_data['Month'] = months
b = pd.get_dummies(all_data['Month'],drop_first=True,prefix="month")
pd.concat([all_data, b['month_2'],b['month_3'],b['month_4'],b['month_5'],b['month_6'],b['month_7'],b['month_8'],b['month_9'],b['month_10'],b['month_11'],b['month_12']], axis=1)
all_data.drop(labels='Month', axis=1)
all_data.drop(labels='Date', axis=1);

In [52]:
# Simple test!!!!
all_data.iloc[[45674]]

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,AssortmentType,NearestCompetitor,Region,Region_AreaKM2,...,Mean_VisibilityKm,Mean_Wind_SpeedKm_h,Min_Dew_PointC,Min_Humidity,Min_Sea_Level_PressurehPa,Min_TemperatureC,Min_VisibilitykM,Precipitationmm,WindDirDegrees,Month
55341,1078,18/01/2018,0,1,1,Super Market,General,355,3,407,...,13.0,19,-1,50,1001,1,10.0,2.03,188,1


## Filling Max_Gust_SpeedKm_h 

In [53]:
all_data['Max_Gust_SpeedKm_h'] = all_data['Max_Gust_SpeedKm_h'].apply(lambda x: 0 if np.isnan(x) else 1);

In [54]:
ratios = (all_data.isnull().sum() / len(all_data))*100
rate = ratios.drop(ratios[ratios == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :rate})
missing_data

,Missing Ratio
Events,25.039665
CloudCover,8.855471


# Looking for outliers(train)

Looking for outliers in the train set

In [55]:
my_train_data = all_data.iloc[0:test_offset,:]

Outliers that are below .5 quantile or above the .95

In [56]:
out1 = {}
for v in my_numerical_variables:
    out1[v] = outliers(my_train_data,.05,.95,v).values
    print("Number of outlier according to .05 and .95 quantile for "+v+" are : "+str(len(out1[v])))
    print("Perc. of outlier w.r.t the whole train "+ str(100*(len(out1[v])/my_train_data.shape[0])))
    print("")

Number of outlier according to .05 and .95 quantile for Min_Dew_PointC are : 30834
Perc. of outlier w.r.t the whole train 7.105295904211928

Number of outlier according to .05 and .95 quantile for Min_Humidity are : 39186
Perc. of outlier w.r.t the whole train 9.029906119947091

Number of outlier according to .05 and .95 quantile for Min_Sea_Level_PressurehPa are : 37247
Perc. of outlier w.r.t the whole train 8.583088686001872

Number of outlier according to .05 and .95 quantile for Min_TemperatureC are : 35677
Perc. of outlier w.r.t the whole train 8.221302522363915

Number of outlier according to .05 and .95 quantile for Min_VisibilitykM are : 19412
Perc. of outlier w.r.t the whole train 4.473243954484075

Number of outlier according to .05 and .95 quantile for Precipitationmm are : 17471
Perc. of outlier w.r.t the whole train 4.025965646445048

Number of outlier according to .05 and .95 quantile for WindDirDegrees are : 21670
Perc. of outlier w.r.t the whole train 4.993570806391403


Just to be sure i normalized the considered data, but as expected the results are the same

In [57]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(my_train_data[my_numerical_variables])
df_normalized_values = scaler.transform(my_train_data[my_numerical_variables])
df_normalized = pd.DataFrame(data = df_normalized_values, columns=my_numerical_variables)
#df_normalized

In [58]:
out_n = {}
for v in my_numerical_variables:
    out_n[v] = outliers(df_normalized,.05,.95,v).values
    print("Number of outlier according to .05 and .95 quantile for "+v+" are : "+str(len(out_n[v])))
    print("Perc. of outlier w.r.t the whole train "+ str(100*(len(out_n[v])/df_normalized.shape[0])))
    print("")

Number of outlier according to .05 and .95 quantile for Min_Dew_PointC are : 30834
Perc. of outlier w.r.t the whole train 7.105295904211928

Number of outlier according to .05 and .95 quantile for Min_Humidity are : 39186
Perc. of outlier w.r.t the whole train 9.029906119947091

Number of outlier according to .05 and .95 quantile for Min_Sea_Level_PressurehPa are : 37247
Perc. of outlier w.r.t the whole train 8.583088686001872

Number of outlier according to .05 and .95 quantile for Min_TemperatureC are : 35677
Perc. of outlier w.r.t the whole train 8.221302522363915

Number of outlier according to .05 and .95 quantile for Min_VisibilitykM are : 19412
Perc. of outlier w.r.t the whole train 4.473243954484075

Number of outlier according to .05 and .95 quantile for Precipitationmm are : 336671
Perc. of outlier w.r.t the whole train 77.58147101793261

Number of outlier according to .05 and .95 quantile for WindDirDegrees are : 21670
Perc. of outlier w.r.t the whole train 4.993570806391403

Outliers that are below .1 quantile or above the .99

In [59]:
out2 = {}
for v in my_numerical_variables:
    out2[v] = outliers(my_train_data,.01,.99,v).values
    print("Number of outlier according to .01 and .99 quantile for "+v+" are : "+str(len(out2[v])))
    print("Perc. of outlier w.r.t the whole train "+ str(100*(len(out2[v])/my_train_data.shape[0])))
    print("")

Number of outlier according to .01 and .99 quantile for Min_Dew_PointC are : 6098
Perc. of outlier w.r.t the whole train 1.4052051120154485

Number of outlier according to .01 and .99 quantile for Min_Humidity are : 5874
Perc. of outlier w.r.t the whole train 1.3535872135091416

Number of outlier according to .01 and .99 quantile for Min_Sea_Level_PressurehPa are : 7670
Perc. of outlier w.r.t the whole train 1.7674521497472107

Number of outlier according to .01 and .99 quantile for Min_TemperatureC are : 5044
Perc. of outlier w.r.t the whole train 1.1623244645795217

Number of outlier according to .01 and .99 quantile for Min_VisibilitykM are : 4261
Perc. of outlier w.r.t the whole train 0.98189225685435

Number of outlier according to .01 and .99 quantile for Precipitationmm are : 3876
Perc. of outlier w.r.t the whole train 0.8931739937966346

Number of outlier according to .01 and .99 quantile for WindDirDegrees are : 4091
Perc. of outlier w.r.t the whole train 0.9427179588808133



Set the outliers found below the .01 quantile and above the .99 quantile to the associated quantile

In [60]:
my_train_data_c = my_train_data.copy()
for v in my_numerical_variables:
    Winsorizing(my_train_data_c,.01,0.99,v)

# Changing basis function and look for correlation

In [27]:
my_train_data_c['NumberOfSales'] = target
my_train_data_c['WindDirDegrees'].head()

0     23.0
1     56.0
2     22.0
3    108.0
5     68.0
Name: WindDirDegrees, dtype: float64

In [42]:
def compute_corr(df,var,degree):
    sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
    new_dataset = df
    current_column = var
    corr = {}
    indexes = []
    for d in range(2,degree+1):
        column_name = var+" at the power of " + str(d)
        new_dataset[column_name] = new_dataset[current_column]*df[var]
       #new_dataset.plot(kind="scatter", x=column_name, y="NumberOfSales")
        corr[column_name] = new_dataset.corr()['NumberOfSales'][column_name]
        print(column_name +"  CORRELATION ::"+str(corr[column_name]))
        current_column = column_name
        indexes.append(current_column)
    
    if (df[var].values > 0).all():
        column_name = var+"logbase2"   
        new_dataset[column_name] = np.log2(df[var])
        indexes.append(current_column)
        #new_dataset.plot(kind="scatter", x=column_name, y="NumberOfSales")
        corr[column_name] = new_dataset.corr()['NumberOfSales'][column_name]
        print(column_name +"  CORRELATION ::"+str(corr[column_name]))
        column_name = var+"logbase10"
        new_dataset[column_name] = np.log10(df[var])
        indexes.append(current_column)
       #new_dataset.plot(kind="scatter", x=column_name, y="NumberOfSales")
        corr[column_name] = new_dataset.corr()['NumberOfSales'][column_name]
        print(column_name +"  CORRELATION ::"+str(corr[column_name]))
    
    return corr,indexes
    #return new_dataset[indexes]

In [43]:
#add_degress returns a new small dataframe for each variable
correlations = {}
indx = {}
for v in my_numerical_variables:
    df = my_train_data_c[[v,'NumberOfSales']].copy()
    correlations[v] , indx[v] = compute_corr(df,v,3)

Min_Dew_PointC at the power of 2  CORRELATION ::-0.040598532860458315
Min_Dew_PointC at the power of 3  CORRELATION ::-0.035715453647362116
Min_Humidity at the power of 2  CORRELATION ::0.011814169101291992
Min_Humidity at the power of 3  CORRELATION ::0.008906455410652648
Min_Humiditylogbase2  CORRELATION ::0.013773256202913059
Min_Humiditylogbase10  CORRELATION ::0.013773256202918477
Min_Sea_Level_PressurehPa at the power of 2  CORRELATION ::-0.004084657895934583
Min_Sea_Level_PressurehPa at the power of 3  CORRELATION ::-0.004072306033114736
Min_Sea_Level_PressurehPalogbase2  CORRELATION ::-0.004111476079502595
Min_Sea_Level_PressurehPalogbase10  CORRELATION ::-0.004111476079499107
Min_TemperatureC at the power of 2  CORRELATION ::-0.03999911280729609
Min_TemperatureC at the power of 3  CORRELATION ::-0.035709916257314914
Min_VisibilitykM at the power of 2  CORRELATION ::0.01233513232030087
Min_VisibilitykM at the power of 3  CORRELATION ::0.0082996505248679
Precipitationmm at the p

In [49]:
for v in my_numerical_variables:
    print(v +"  LINEAR CORRELATION ::"+str(my_train_data_c.corr()['NumberOfSales'][v]))
    for k in indx[v]:
        print(v +"  Difference in CORRELATION btw + "+ v +" and "+ k +" :: "+str(np.absolute(my_train_data_c.corr()['NumberOfSales'][v] - correlations[v][k])))

Min_Dew_PointC  LINEAR CORRELATION ::-0.04186373501690499
Min_Dew_PointC  Difference in CORRELATION btw + Min_Dew_PointC and Min_Dew_PointC at the power of 2::0.001265202156446675
Min_Dew_PointC  Difference in CORRELATION btw + Min_Dew_PointC and Min_Dew_PointC at the power of 3::0.006148281369542874
Min_Humidity  LINEAR CORRELATION ::0.013635576424895882
Min_Humidity  Difference in CORRELATION btw + Min_Humidity and Min_Humidity at the power of 2::0.0018214073236038908
Min_Humidity  Difference in CORRELATION btw + Min_Humidity and Min_Humidity at the power of 3::0.004729121014243234
Min_Humidity  Difference in CORRELATION btw + Min_Humidity and Min_Humidity at the power of 3::0.004729121014243234
Min_Humidity  Difference in CORRELATION btw + Min_Humidity and Min_Humidity at the power of 3::0.004729121014243234
Min_Sea_Level_PressurehPa  LINEAR CORRELATION ::-0.004097714730857049
Min_Sea_Level_PressurehPa  Difference in CORRELATION btw + Min_Sea_Level_PressurehPa and Min_Sea_Level_Pres